In [57]:
#r "nuget:FParsec"
open FParsec

let loadFile fname = System.IO.File.ReadAllText(fname)

let sample = "day3.sample.txt"
let data = "day3.data.txt"

let str = pstring

Installed Packages FParsec, 1.1.1

In [58]:
let numPositionParser = tuple3 getPosition pint32 getPosition
let skipNonDigit = skipManySatisfy (Char.IsDigit >> not)
let enginePartParser = skipNonDigit >>. numPositionParser .>> skipNonDigit
let fileParser = many enginePartParser

let parseEngineParts (s: string) =
    match run fileParser s with
    | Success (result, _, _) -> result
    | Failure (message, _, _) -> failwith message

let testString = loadFile(sample)
let testResult = parseEngineParts testString
testResult


[ ((Ln: 1, Col: 1), 467, (Ln: 1, Col: 4)), ((Ln: 1, Col: 6), 114, (Ln: 1, Col: 9)), ((Ln: 3, Col: 3), 35, (Ln: 3, Col: 5)), ((Ln: 3, Col: 7), 633, (Ln: 3, Col: 10)), ((Ln: 5, Col: 1), 617, (Ln: 5, Col: 4)), ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)), ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) ] HeadOrDefault ((Ln: 1, Col: 1), 467, (Ln: 1, Col: 4)) Item1 (Ln: 1, Col: 1) Index 0 Line 1 Column 1 StreamName Item2 467 Item3 (Ln: 1, Col: 4) Index 3 Line 1 Column 4 StreamName TailOrNull [ ((Ln: 1, Col: 6), 114, (Ln: 1, Col: 9)), ((Ln: 3, Col: 3), 35, (Ln: 3, Col: 5)), ((Ln: 3, Col: 7), 633, (Ln: 3, Col: 10)), ((Ln: 5, Col: 1), 617, (Ln: 5, Col: 4)), ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)), ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) ] HeadOrDefault ((Ln: 1, Col: 6), 114, (Ln: 1, Col: 9)) Item1 (Ln: 1, Col: 6) Index 5 Line 1 Column 6 StreamName Item2 114 Item3 (Ln: 1, Col: 9) Index 8 Line 1 Column 9 StreamName TailOrNull [ ((Ln: 3, Col: 3), 35, (Ln: 3, Col: 5)), ((Ln: 3, Col: 7), 633, (Ln: 3, Col: 10)), ((Ln: 5, Col: 1), 617, (Ln: 5, Col: 4)), ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)), ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) ] HeadOrDefault ((Ln: 3, Col: 3), 35, (Ln: 3, Col: 5)) Item1 (Ln: 3, Col: 3) Index 24 Line 3 Column 3 StreamName Item2 35 Item3 (Ln: 3, Col: 5) Index 26 Line 3 Column 5 StreamName TailOrNull [ ((Ln: 3, Col: 7), 633, (Ln: 3, Col: 10)), ((Ln: 5, Col: 1), 617, (Ln: 5, Col: 4)), ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)), ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) ] HeadOrDefault ((Ln: 3, Col: 7), 633, (Ln: 3, Col: 10)) Item1 (Ln: 3, Col: 7) Index 28 Line 3 Column 7 StreamName Item2 633 Item3 (Ln: 3, Col: 10) Index 31 Line 3 Column 10 StreamName TailOrNull [ ((Ln: 5, Col: 1), 617, (Ln: 5, Col: 4)), ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)), ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) ] HeadOrDefault ((Ln: 5, Col: 1), 617, (Ln: 5, Col: 4)) Item1 (Ln: 5, Col: 1) Item2 617 Item3 (Ln: 5, Col: 4) TailOrNull [ ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)), ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) ] HeadOrDefault ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)) TailOrNull [ ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) ] Head ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)) Tail [ ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) ] (values) index value 0 ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)) 1 ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)) 2 ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)) 3 ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)) 4 ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) Head ((Ln: 5, Col: 1), 617, (Ln: 5, Col: 4)) Item1 (Ln: 5, Col: 1) Item2 617 Item3 (Ln: 5, Col: 4) Tail [ ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)), ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 598, (Ln: 10, Col: 9)) ] HeadOrDefault ((Ln: 6, Col: 8), 58, (Ln: 6, Col: 10)) TailOrNull [ ((Ln: 7, Col: 3), 592, (Ln: 7, Col: 6)), ((Ln: 8, Col: 7), 755, (Ln: 8, Col: 10)), ((Ln: 10, Col: 2), 664, (Ln: 10, Col: 5)), ((Ln: 10, Col: 6), 59

In [61]:
type Number = {
    Number : int
    Row : int
    Col : int
    Length : int
}


module Number =
    let fromParsedData (startPos : Position, num : int, endPos : Position) : Number =
        {
            Number = num
            Row = int startPos.Line - 1
            Col = int startPos.Column - 1
            Length = int endPos.Column - int startPos.Column
        }

    let isEnginePart (dataRows : string array) (n : Number) =
        let w = dataRows[0].Length
        let h = dataRows.Length
        let neighborPositions =
            [
                // left neighbor
                (n.Col - 1, n.Row)
                // right neighbor
                (n.Col + n.Length, n.Row)
                // rowAbove
                yield! seq {(n.Col - 1)..(n.Col+n.Length)} |> Seq.map (fun c -> (c, n.Row-1))
                // rowBelow
                yield! seq {(n.Col - 1)..(n.Col+n.Length)} |> Seq.map (fun c -> (c, n.Row+1))
            ]
            |> List.filter (fun (col, row) ->
                row >= 0 && row + 1 <= w && col >= 0 && col + 1 <= h
                )

        neighborPositions
        |> List.map (fun (col, row) -> dataRows[row][col])
        //|> List.exists ((<>) '.')

    

let numbers = testResult |> List.map Number.fromParsedData 
let dataRows = testString.Split('\n')

numbers |> List.map (Number.isEnginePart dataRows)




[ [ ., ., ., ., * ], [ ., ., b, ., ., ., . ], [ ., ., ., ., *, b, ., ., ., . ], [ ., ., ., ., ., ., ., ., #, ., ., . ], [ *, ., ., ., ., ., ., ., . ], [ ., ., ., ., ., ., ., ., ., . ], [ ., ., ., ., ., ., +, ., ., ., ., . ], [ ., ., ., ., ., ., ., *, ., ., ., . ], [ ., ., ., ., ., $, . ], [ ., ., ., *, ., ., . ] ] HeadOrDefault [ ., ., ., ., * ] HeadOrDefault . TailOrNull [ ., ., ., * ] HeadOrDefault . TailOrNull [ ., ., * ] HeadOrDefault . TailOrNull [ ., * ] HeadOrDefault . TailOrNull [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] Head . Tail [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] (values) [ ., * ] Head . Tail [ ., * ] HeadOrDefault . TailOrNull [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] Head . Tail [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] (values) [ ., * ] (values) [ ., ., * ] Head . Tail [ ., ., * ] HeadOrDefault . TailOrNull [ ., * ] HeadOrDefault . TailOrNull [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] Head . Tail [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] (values) [ ., * ] Head . Tail [ ., * ] HeadOrDefault . TailOrNull [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] Head . Tail [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] (values) [ ., * ] (values) [ ., ., * ] (values) [ ., ., ., * ] Head . Tail [ ., ., ., * ] HeadOrDefault . TailOrNull [ ., ., * ] HeadOrDefault . TailOrNull [ ., * ] HeadOrDefault . TailOrNull [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] Head . Tail [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] (values) [ ., * ] Head . Tail [ ., * ] HeadOrDefault . TailOrNull [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] Head . Tail [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] (values) [ ., * ] (values) [ ., ., * ] Head . Tail [ ., ., * ] HeadOrDefault . TailOrNull [ ., * ] HeadOrDefault . TailOrNull [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] Head . Tail [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] (values) [ ., * ] Head . Tail [ ., * ] HeadOrDefault . TailOrNull [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] Head . Tail [ * ] HeadOrDefault * TailOrNull [ ] Head * Tail [ ] (values) [ * ] (values) [ ., * ] (values) [ ., ., * ] (values) [ ., ., ., * ] (values) [ ., ., ., ., * ] TailOrNull [ [ ., ., b, ., ., ., . ], [ ., ., ., ., *, b, ., ., ., . ], [ ., ., ., ., ., ., ., ., #, ., ., . ], [ *, ., ., ., ., ., ., ., . ], [ ., ., ., ., ., ., ., ., ., . ], [ ., ., ., ., ., ., +, ., ., ., ., . ], [ ., ., ., ., ., ., ., *, ., ., ., . ], [ ., ., ., ., ., $, . ], [ ., ., ., *, ., ., . ] ] HeadOrDefault [ ., ., b, ., ., ., . ] HeadOrDefault . TailOrNull [ ., b, ., ., ., . ] HeadOrDefault . TailOrNull [ b, ., ., ., . ] HeadOrDefault b TailOrNull [ ., ., ., . ] HeadOrDefault . TailOrNull [ ., ., . ] Head . Tail [ ., ., . ] (values) [ ., ., ., . ] Head b Tail [ ., ., ., . ] HeadOrDefault . TailOrNull [ ., ., . ] Head . Tail [ ., ., . ] (values) [ ., ., ., . ] (values) [ b, ., ., ., . ] Head . Tail [ b, ., ., ., . ] HeadOrDefault b TailOrNull [ ., ., ., . ] HeadOrDefault . TailOrNull [ ., ., . ] Head . Tail [ ., ., . ] (values) [ ., ., ., . ] Head b Tail [ ., ., ., . ] HeadOrDefault . TailOrNull [ ., ., . ] Head . Tail [ ., ., . ] (values) [ ., ., ., . ] (values) [ b, ., ., ., . ] (values) [ ., b, ., ., ., . ] Head . Tail [ ., b, ., ., ., . ] HeadOrDefault . TailOrNull [ b, ., ., ., . ] HeadOrDefault b TailOrNull [ ., ., ., . ] HeadOrDefault . TailOrNull [ ., ., . ] Head . Tail [ ., ., . ] (values) [ ., ., ., . ] Head b Tail [ ., ., ., . ] HeadOrDefault . TailOrNull [ ., ., . ] Head . Tail [ ., ., . ] (values) [ ., ., ., . ] (values) [ b, ., ., ., . ] Head . Tail [ b, ., ., ., . ] HeadOrDefault b TailOrNull [ ., ., ., . ] HeadOrDefault . TailOrNull [ ., ., . ] Head . Tail [ ., ., . ] (val